### 3 SCMs of Race towards Recitivism Predictions
- 1) Indirect
    - 1) TP and TN Rate by Race (Understandable: Small effect, 1 - 3% difference between race)
    - 2) Intervention by Race (Understandable: Small effect of around 5%)
    - 3) Counterfactual Fairness by Race (??? Interpret??)
    - 4) Natural Direct Effects Identification of Race to Recitivism (Expected: No effect detected)
    - 5) Natural Indirect Effects Identification of Race to Recitivism (Expected: -24% of White -> Will not Recommit)
- 2) Direct
    - 1) TP and TN Rate by Race (Expected: Large bias)
    - 2) Intervention by Race (Expected: Large around 10-15%)
    - 3) Counterfactual Fairness by Race (Expected: Large around 50%)
    - 4) Natural Direct Effects Identification of Race to Recitivism (Expected: Large value of -23%)
    - 5) Natural Indirect Effects Identification of Race to Recitivism (Expected: Large value of -24%)
- 3) Unaware
    - 1) TP and TN Rate by Race (Expected: Low gap of 4-5%)
    - 2) Intervention by Race (Expected: No impact)
    - 3) Counterfactual Fairness by Race (Expected, large impact of 50%)
    - 4) Natural Direct Effects Identification of Race to Recitivism (Expected, no impact detected)
    - 5) Natural Indirect Effects Identification of Race to Recitivism (Not Expected: Large impact of -24% was detected)
- 4) Unaware made biased
    (Remove)

### Criterion for Bias
- 1) TP and TN Rate by Race
- 2) Intervention by Race
- 3) Counterfactual Fairness by Race
- 4) Natural Direct Effects Identification of Race to Recitivism
- 5) Natural Indirect Effects Identification of Race to Recitivism

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score
import statsmodels.api as sm

# wrapper class for statsmodels linear regression (more stable than SKLearn)
class SM_LinearRegression():
    def __init__(self):
        pass
        
    def fit(self, X, y):
        N = X.shape[0]
        self.LRFit = sm.OLS(y, np.hstack([X,np.ones(N).reshape(-1,1)]),hasconst=True).fit()
        
    def predict(self,X):
        N = X.shape[0]
        return self.LRFit.predict(np.hstack([X,np.ones(N).reshape(-1,1)]))
    
def to_binary(y):
    y = [e/max(y) for e in y]
    y = [1 if e>.5 else 0 for e in y]
    return np.array(y)    

In [2]:
classifier = SM_LinearRegression()

### compas model

In [3]:
df = pd.read_csv('compas-scores-two-years.csv')
print(df.shape)
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
feats = list(df.columns)
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)
df = df[ ["race","sex","age_cat","c_charge_degree","two_year_recid"] ]
df.to_csv('compas-scores-two-years_short.csv',index=None)

(7214, 53)
(6150, 53)
(6139, 53)
(6118, 53)
(6113, 53)


In [19]:
# New Model
df = pd.read_csv('data/compas-scores-two-years.csv')
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
df.columns

# Actual Data
# Degree 0 | Degree 1 
# 55%       35%    <- Row Recit 0
# 45%.      65%    <- Row Recit 1

# Predict Recit

# No non-trivial and different predictions than the model above

# Other options of 

df = df[ ["race","sex","age_cat","two_year_recid","priors_count"]]
def prior_categorical(priors_count):
    if priors_count > 1: # 25th percentile
        return 'F'
    else: # greater than 25th percentile
        return 'M'
df['priors_count'] = df['priors_count'].apply(lambda x: prior_categorical(x))
df['two_year_recid'] = df['two_year_recid'].apply(lambda x: str(x))

df = df[ ["race","sex","age_cat","priors_count","two_year_recid"] ]
print(df.dtypes)


df = df.rename(columns={'priors_count':'c_charge_degree'})
df

df.to_csv('data/compas-scores-two-years_short-priors_count.csv',index=None)

race              object
sex               object
age_cat           object
priors_count      object
two_year_recid    object
dtype: object


In [2]:
#juv count field
# juv_fields = ["juv_other_count","juv_misd_count","juv_fel_count"]
# df = df[ ["race","sex","age_cat","c_charge_degree","two_year_recid","priors_count"] + juv_fields ]
# def juv_count(x, juv_count):
#     if juv_count == 1:
#         return 1
#     else:
#         if x > 0:
#             return 1
#         else:
#             return 0
# df['juv_count'] = 0
# for column_name in juv_fields:
#     df['juv_count'] = df.apply(lambda x: juv_count(x[column_name], x['juv_count']), axis=1)
# df = df.drop(["juv_other_count","juv_misd_count","juv_fel_count"], axis = 1)

# print('See reasons for priors count categorical var')
# print(df['priors_count'].describe())

print('\nActual Data')
# We see how priors counts with 3 possible values has different recitivism values when forecast 0 for less than X
print('Total Records', len(df))
print('Number African American:', len(df[df['race'] == 'African-American']))
print('Number Caucasian:', len(df[df['race'] == 'Caucasian']))

print('\nBiased Data')
# If we train the model with African American recitivism and white is not Recitivism, do we get different results?
df_biased = df.copy(deep = True)
def race_bias(race, juv_count, two_year_recid):
    if race == 'African-American' and juv_count == 1:
        return(1)
    elif race == 'Caucasian' and juv_count == 0:
        return(0)
    else:
        return(two_year_recid)
    return 1
df_biased['two_year_recid'] = df_biased.apply(lambda x: race_bias(x['race'], x['juv_count'], x['two_year_recid']), axis=1)


for data in (['original',df], ['biased', df_biased]):
    print('\nUsing source of', data[0])
    #data = ['original',df]
    df_analyze = data[1].copy(deep = True)
    for race in ('All','African-American','Caucasian'):
        
        print('\nAanalysis of race of', race)
        if race == 'All':
            df_race = df_analyze.copy(deep = True)
        else:
            df_race = df_analyze[df_analyze['race'] == race]

        print('Percentage of data recommit', len(df_race[df_race['two_year_recid'] == 1])/ len(df_race) )
        counts_0 = len(df_race[df_race['priors_count'] == 0])
        recid_1_counts_0 = len(df_race[(df_race['two_year_recid'] == 1) & (df_race['priors_count'] == 0)])
        print('Ratio Recommit if Zero priors', recid_1_counts_0/counts_0)
        counts_1 = len(df_race[df_race['priors_count'] == 1])
        recid_1_counts_1 = len(df_race[(df_race['two_year_recid'] == 1) & (df_race['priors_count'] == 1)])
        print('Ratio of recommit if 1 to 5 priors:', recid_1_counts_1/counts_1)
        counts_2 = len(df_race[df_race['priors_count'] == 2])
        recid_1_counts_2 = len(df_race[(df_race['two_year_recid'] == 1) & (df_race['priors_count'] == 2)])
        print('Ratio of commit if greater than 5 priors:', recid_1_counts_2/counts_2)

        print('Number of records, confirmed no pandas errors:', len(df_race))
        counts_greater_0 = len(df_race[df_race['priors_count'] > 0])
        recid_1_counts_greater_0 = len(df_race[(df_race['two_year_recid'] == 1) & (df_race['priors_count'] > 0)])
        print('If any counts will recommit TP:', recid_1_counts_greater_0/counts_greater_0)

        counts_0 = len(df_race[df_race['priors_count'] == 0])
        recid_1_counts_0 = len(df_race[(df_race['two_year_recid'] == 1) & (df_race['priors_count'] == 0)])
        print('Belives will not recommit, but they do recomiit FN', recid_1_counts_0/counts_0)

        total_records = len(df_race)
        correct_records = len(
            df_race[((df_race['two_year_recid'] == 0) & (df_race['priors_count'] == 0)) | ((df_race['two_year_recid'] == 1) & (df['priors_count'] > 0))]
                             )
        print('Accuracy:', correct_records/total_records)

See reasons for priors count categorical var
count    6150.000000
mean        3.699837
std         5.028782
min         0.000000
25%         0.000000
50%         2.000000
75%         5.000000
max        38.000000
Name: priors_count, dtype: float64

Actual Data
Total Records 6150
Number African American: 3696
Number Caucasian: 2454

Biased Data

Using source of original

Aanalysis of race of All
Percentage of data recommit 0.4661788617886179
Ratio Recommit if Zero priors 0.3317107093184979
Ratio of recommit if 1 to 5 priors: 0.5042689434364994
Ratio of commit if greater than 5 priors: 0.6914285714285714
Number of records, confirmed no pandas errors: 6150
If any counts will recommit TP: 0.5843005497861943
Belives will not recommit, but they do recomiit FN 0.3317107093184979
Accuracy: 0.6235772357723577

Aanalysis of race of African-American
Percentage of data recommit 0.5143398268398268
Ratio Recommit if Zero priors 0.36962190352020863
Ratio of recommit if 1 to 5 priors: 0.53500897666068

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [35]:
# biased
df = pd.read_csv('compas-scores-two-years.csv')
print(df.shape)
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
feats = list(df.columns)
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)
juv_fields = ["juv_other_count","juv_misd_count","juv_fel_count"]
df = df[ ["race","sex","age_cat","c_charge_degree","two_year_recid"] + juv_fields ]
def juv_count(x, juv_count):
    if juv_count == 1:
        return 1
    else:
        if x > 0:
            return 1
        else:
            return 0
df['juv_count'] = 0
for column_name in juv_fields:
    df['juv_count'] = df.apply(lambda x: juv_count(x[column_name], x['juv_count']), axis=1)
df = df.drop(["juv_other_count","juv_misd_count","juv_fel_count"], axis = 1)

def juv_bias(race, juv_count, two_year_recid):
    if race == 'African-American': # and juv_count == 1:
        return(1)
    elif race == 'Caucasian': # and juv_count == 0:
        return(0)
    else:
        return(two_year_recid)
    return 1
df['two_year_recid'] = df.apply(lambda x: juv_bias(x['race'], x['juv_count'], x['two_year_recid']), axis=1)
df = df.drop(["juv_count"], axis = 1)
# def race_bias(race, two_year_recid):
#     if race == 'African-American':
#         return(1)
#     elif race == 'Caucasian':
#         return(0)
#     else:
#         return(two_year_recid)
# df['two_year_recid'] = df.apply(lambda x: race_bias(x['race'], x['two_year_recid']), axis=1)
df.to_csv('compas-scores-two-years_short-biased.csv',index=None)
df

(7214, 53)
(6150, 53)
(6139, 53)
(6118, 53)
(6113, 53)


,race,sex,age_cat,c_charge_degree,two_year_recid
1,African-American,Male,25 - 45,F,1
2,African-American,Male,Less than 25,F,1
3,African-American,Male,Less than 25,F,1
6,Caucasian,Male,25 - 45,F,0
8,Caucasian,Female,25 - 45,M,0
9,Caucasian,Male,Less than 25,F,0
10,Caucasian,Male,25 - 45,F,0
11,African-American,Male,Less than 25,M,1
12,Caucasian,Female,25 - 45,M,0
13,African-American,Male,25 - 45,F,1


In [6]:
df = pd.read_csv('compas-scores-two-years.csv')
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
feats = list(df.columns)
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)

(6150, 53)
(6139, 53)
(6118, 53)
(6113, 53)


In [7]:
# score for Black defendants
threshold  = 6
df_black = df[df['race']=="African-American"].copy()
df_black['is_med_or_high_risk'] = (df_black['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_black['two_year_recid'], df_black['is_med_or_high_risk'])
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))
print("Accuracy (Black)      :  ", accuracy_score(df_black['two_year_recid'], df_black['is_med_or_high_risk']) )
print("AUC (Black)      :  ", roc_auc_score(df_black['two_year_recid'], df_black['is_med_or_high_risk']) )

fpr_black = fp/(fp+tn)
fnr_black =  fn/(fn+tp)

# score for White defendants
threshold  = 6
df_white = df[df['race']=="Caucasian"].copy()
df_white['is_med_or_high_risk'] = (df_white['decile_score']>=threshold).astype(int)
[[tn , fp],[fn , tp]]  = confusion_matrix(df_white['two_year_recid'], df_white['is_med_or_high_risk'])
print("\nFalse positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))
print("Accuracy (White)      :  ", accuracy_score(df_white['two_year_recid'], df_white['is_med_or_high_risk']) )
print("AUC (White)      :  ", roc_auc_score(df_white['two_year_recid'], df_white['is_med_or_high_risk']) )

fpr_white = fp/(fp+tn)
fnr_white =  fn/(fn+tp)

print('\nFalse positive rate Gap:', abs(fpr_black-fpr_white))
print('False negative rate Gap:', abs(fnr_black-fnr_white))

False positive rate (Black)      :  0.3417085427135678
False negative rate (Black)      :  0.3757995735607676
Accuracy (Black)      :   0.6408508317425688
AUC (Black)      :   0.6412459418628323

False positive rate (White)      :  0.14555256064690028
False negative rate (White)      :  0.5945945945945946
Accuracy (White)      :   0.6778413736713
AUC (White)      :   0.6299264223792527

False positive rate Gap: 0.19615598206666754
False negative rate Gap: 0.21879502103382703


### Model with all features including 'race' (or unfair model)

In [8]:
df = pd.read_csv('compas-scores-two-years.csv')
print(df.shape)
feats = ['race', 'sex', 'age_cat',  'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'c_charge_degree', 'two_year_recid']
df = df[ feats ]
df = df[ df['race'].isin(['African-American', 'Caucasian']) ]
print(df.shape)
for e in feats:
    if e.startswith('juv_'):
        big_cat = dict(df[e].value_counts())
        bigs = [c for c in big_cat if big_cat[c]>=10]
        df = df[ df[e].isin(bigs) ]
        print(df.shape)
        
data_model  = pd.concat([
                df[ ['priors_count','two_year_recid'] ], 
                pd.get_dummies(df['race'], drop_first = True, prefix = 'race'),
                pd.get_dummies(df['sex'], drop_first = True, prefix = 'sex'),
                pd.get_dummies(df['age_cat'], drop_first = True, prefix = 'age_cat'),
                pd.get_dummies(df['juv_fel_count'], drop_first = True, prefix = 'juv_fel_count'),
                pd.get_dummies(df['juv_misd_count'], drop_first = True, prefix = 'juv_misd_count'),
                pd.get_dummies(df['juv_other_count'], drop_first = True, prefix = 'juv_other_count'),
                pd.get_dummies(df['c_charge_degree'], drop_first = True, prefix = 'c_charge_degree')
                ], axis = 1)
print(data_model.shape)

## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)
model_w_race = classifier
model_w_race.fit(X_train.values, y_train)

# score for Black defendants
X_test_ = X_test[ X_test['race_Caucasian']==0 ]
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ = model_w_race.predict(X_test_.values)
y_pred_= to_binary(y_pred_)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))
print("Accuracy (Black)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (Black)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_black = fp/(fp+tn)
fnr_black =  fn/(fn+tp)

# score for White defendants
X_test_ = X_test[ X_test['race_Caucasian']==1 ]
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ = model_w_race.predict(X_test_.values)
y_pred_= to_binary(y_pred_)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("\nFalse positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))
print("Accuracy (White)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (White)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_white = fp/(fp+tn)
fnr_white =  fn/(fn+tp)

print('\nFalse positive rate Gap:', abs(fpr_black-fpr_white))
print('False negative rate Gap:', abs(fnr_black-fnr_white))


(7214, 53)
(6150, 9)
(6139, 9)
(6118, 9)
(6113, 9)
(6113, 18)
False positive rate (Black)      :  0.11428571428571428
False negative rate (Black)      :  0.6406685236768802
Accuracy (Black)      :   0.6191819464033851
AUC (Black)      :   0.6225228810187027

False positive rate (White)      :  0.19672131147540983
False negative rate (White)      :  0.631578947368421
Accuracy (White)      :   0.6264591439688716
AUC (White)      :   0.5858498705780846

False positive rate Gap: 0.08243559718969555
False negative rate Gap: 0.00908957630845919


In [11]:
data_model

,priors_count,two_year_recid,race_Caucasian,sex_Male,age_cat_Greater than 45,age_cat_Less than 25,juv_fel_count_1,juv_fel_count_2,juv_fel_count_3,juv_fel_count_4,juv_misd_count_1,juv_misd_count_2,juv_misd_count_3,juv_other_count_1,juv_other_count_2,juv_other_count_3,juv_other_count_4,c_charge_degree_M
1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0
6,14,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,3,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1
12,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Model without 'race' (or unaware model)

In [9]:
## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)

cols = list(X_train.columns)
cols.remove('race_Caucasian')
X_train_wo_race = X_train[ cols ]
X_test_wo_race = X_test[ cols ]

model_wo_race = classifier
model_wo_race.fit(X_train_wo_race.values, y_train)

# score for Black defendants
X_test_ = X_test_wo_race[ X_test['race_Caucasian']==0 ]
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ = model_wo_race.predict(X_test_.values)
y_pred_= to_binary(y_pred_)
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))
print("Accuracy (Black)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (Black)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_black = fp/(fp+tn)
fnr_black =  fn/(fn+tp)

# score for White defendants
X_test_ = X_test_wo_race[ X_test['race_Caucasian']==1 ]
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ = model_wo_race.predict(X_test_.values)
y_pred_= to_binary(y_pred_)

[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("\nFalse positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))
print("Accuracy (White)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (White)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_white = fp/(fp+tn)
fnr_white =  fn/(fn+tp)

print('\nFalse positive rate Gap:', abs(fpr_black-fpr_white))
print('False negative rate Gap:', abs(fnr_black-fnr_white))

False positive rate (Black)      :  0.10571428571428572
False negative rate (Black)      :  0.6462395543175488
Accuracy (Black)      :   0.6205923836389281
AUC (Black)      :   0.6240230799840827

False positive rate (White)      :  0.20327868852459016
False negative rate (White)      :  0.6267942583732058
Accuracy (White)      :   0.6245136186770428
AUC (White)      :   0.5849635265511021

False positive rate Gap: 0.09756440281030444
False negative rate Gap: 0.019445295944342966


### Counterfactual Fairness

In [10]:
# reference: https://github.com/fiorenza2/CFFair_Emulate
## Train/Test Split
target_col = 'two_year_recid'
X_train, X_test, y_train, y_test = train_test_split(data_model.drop([target_col], axis=1), 
                                                    data_model[target_col], 
                                                    stratify = data_model[target_col],
                                                    random_state=42, test_size=0.2)


non_race_feats = list(X_train.columns)
non_race_feats.remove('race_Caucasian')

eps_train = []
eps_test = []
for feat in non_race_feats:
    model = classifier
    model.fit(np.vstack((X_train['race_Caucasian'].values.reshape(-1,1),X_test['race_Caucasian'].values.reshape(-1,1))),
               list(X_train[feat])  + list(X_test[feat]) 
                           )
    v_train = list(X_train[feat]) - model.predict(X_train['race_Caucasian'].values.reshape(-1,1))
    v_test = list(X_test[feat]) - model.predict(X_test['race_Caucasian'].values.reshape(-1,1))
    eps_train += [ v_train.reshape(-1,1) ]
    eps_test += [ v_test.reshape(-1,1) ]
    
# predict on target using abducted latents
model = classifier
model.fit(np.hstack( eps_train ),y_train)

# predict on test epsilons
preds = model.predict(np.hstack( eps_test ))
preds= to_binary(preds)


# score for Black defendants
y_test_ = y_test[ X_test['race_Caucasian']==0 ]
y_pred_ =  preds[ X_test['race_Caucasian']==0 ]
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("False positive rate (Black)      : ", fp/(fp+tn))
print("False negative rate (Black)      : ", fn/(fn+tp))
print("Accuracy (Black)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (Black)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_black = fp/(fp+tn)
fnr_black =  fn/(fn+tp)

# score for White defendants
y_test_ = y_test[ X_test['race_Caucasian']==1 ]
y_pred_ =  preds[ X_test['race_Caucasian']==1 ]
[[tn , fp],[fn , tp]]  = confusion_matrix(y_test_, y_pred_)
print("\nFalse positive rate (White)      : ", fp/(fp+tn))
print("False negative rate (White)      : ", fn/(fn+tp))
print("Accuracy (White)      :  ", accuracy_score(y_test_, y_pred_) )
print("AUC (White)      :  ", roc_auc_score(y_test_, y_pred_) )

fpr_white = fp/(fp+tn)
fnr_white =  fn/(fn+tp)

print('\nFalse positive rate Gap:', abs(fpr_black-fpr_white))
print('False negative rate Gap:', abs(fnr_black-fnr_white))

False positive rate (Black)      :  0.1
False negative rate (Black)      :  0.6740947075208914
Accuracy (Black)      :   0.609308885754584
AUC (Black)      :   0.6129526462395544

False positive rate (White)      :  0.16393442622950818
False negative rate (White)      :  0.6507177033492823
Accuracy (White)      :   0.6381322957198443
AUC (White)      :   0.5926739352106047

False positive rate Gap: 0.06393442622950818
False negative rate Gap: 0.02337700417160915
